# Desafio - Aula 3

## 1) Crie uma classe de tratamento de texto. A classe deverá conter:
   ## -> remoção de números
   ## -> passar o texto para caixa baixa
   ## -> remoção de caracteres especiais
   ## -> remoção de stop-words
   ## -> Stemização/lemmatização (deve ser passado como parâmetro qual abordagem utilizar)

In [66]:
!pip install spacy
!python -m spacy download pt_core_news_sm
!python -m spacy download en

You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


[+] Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
symbolic link created for C:\ProgramData\Anaconda3\lib\site-packages\spacy\data\en <<===>> C:\ProgramData\Anaconda3\lib\site-packages\en_core_web_sm
[+] Linking successful
C:\ProgramData\Anaconda3\lib\site-packages\en_core_web_sm -->
C:\ProgramData\Anaconda3\lib\site-packages\spacy\data\en
You can now load the model via spacy.load('en')


You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [78]:
import re
import nltk
import spacy
import nltk
from nltk.stem import PorterStemmer
import pandas as pd
from nltk.stem.rslp import RSLPStemmer

STEM = "Stemizacao"
LEM = "Lammatizacao"
    
class Clean_Text:
    
    def __init__(self, lan = 'english', load_v = None, clean_regex = "([aA-zZ]+)", stemmer = PorterStemmer()):
        nltk.download('stopwords')
        self.regex = clean_regex
        self.stops = nltk.corpus.stopwords.words(lan)
        if load_v == None:
            if lan == "english":
                self.nlp = spacy.load('en')
            elif lan == "portuguese":
                self.nlp = spacy.load('pt_core_news_sm')
            else:
                raise("idioma invalido")
        else:
            self.nlp = spacy.load(load_v)
        self.ps = stemmer
        
    def clean(self, df, norm = None, stops = None):
        if (norm != STEM) and (norm != LEM) and (norm != None):
            raise("Normalizacao invalida")
        
        if stops != None:
            self.stops = stops
            
        return df.apply(self.remove_special_char).apply(lambda x: self.remove_stop_words(x, self.stops)).apply(lambda x: self.normalize(x, norm))
        
    def remove_special_char(self, text):
        return " ".join(["".join(re.findall(self.regex, word)).lower() for word in text.split(" ")])
    
    def remove_stop_words(self, text, stops):
        if stops != None:
            return " ".join([word if word not in self.stops else "" for word in text.split(" ")])
        else:
            return text
        
    def normalize(self, text, norm):
        if norm == LEM:
            doc = self.nlp(text)
            return " ".join([token.lemma_ for token in doc])
        elif norm == STEM:
            
            return " ".join([self.ps.stem(word) for word in text.split(" ")])
        else:
            return text
                
d = {"a": ["I World1!.@ é223 dogs daspd% did conected", "hell0o world!"]}
df = pd.DataFrame(d)
clean_text = Clean_Text(lan = "chi")
clean_text.clean(df['a'], STEM)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\logonrmlocal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


OSError: No such file or directory: 'C:\\Users\\logonrmlocal\\AppData\\Roaming\\nltk_data\\corpora\\stopwords\\chi'

In [2]:
nlp = spacy.load('pt_core_news_sm')
doc = nlp(u'quebrou,quebraram,quebrado,quebrariam')
[token.lemma_ for token in doc if token.pos_ == 'VERB']

['quebrar', 'quebrar', 'quebrar', 'quebrar']

## 2) Refaça o exercício de aula (movie_review) realizando o tratamento do texto antes. Houve diferença? Descreva

In [ ]:
# resposta 2

## 3) Utilizando o dataset disposto no portal, faça:
   ## -> extraia o dataset na pasta do notebook
   ## -> crie uma função que leia o conteúdo de cada uma das pastas, amazene num dataframe com duas colunas (review, tag)
   ## -> utilize a classe de tratamento de texto criada acima para tratar o texto
   ## -> crie um pipeline de classificação de texto (countvectorizer/tfidfvectorizer,divisão em treino/teste,instância de modelo, fit e predict)
   ## -> imprima o relatório de classificação
   ## -> OBS: teste várias opções de stemming/lemmatizer

In [ ]:
# resposta 3